In [80]:
import yfinance as yf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import requests

### Question 1: Withdrawn IPOs by Company Type
What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list [stockanalysis.com/ipos/withdrawn](stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Definition of terms
* **IPO withdrawal:** In the world of stock analytics, this refers to a situation where a company that has filed to go public (i.e., launched an Initial Public Offering) decides to cancel or postpone the offering before its shares are officially listed and traded on a stock exchange.

In [46]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/withdrawn/"
response = requests.get(url, headers=headers)

ipo_withdraw = pd.read_html(response.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\3236582231.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_withdraw = pd.read_html(response.text)


In [47]:
ipo_withdraw_dfs = ipo_withdraw[0]
ipo_withdraw_dfs.head(2)

,Symbol,Company Name,Price Range,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
1,AURN,"Aurion Biotech, Inc.",-,-


In [48]:
ipo_withdraw_dfs.loc[0]

Symbol                              UNFL
Company Name      Unifoil Holdings, Inc.
Price Range                $3.00 - $4.00
Shares Offered                   2000000
Name: 0, dtype: object

In [49]:
ipo_withdraw_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          99 non-null     object
 1   Company Name    99 non-null     object
 2   Price Range     99 non-null     object
 3   Shares Offered  99 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [58]:
# IPO value will the thw average of price range * shared offer
def split_price_range(x):
    if x == '-' or pd.isna(x):
        return 0.0
    
    x = x.replace('$', '').replace(' ', '')
    if '-' in x:
        low, high = map(float, x_lst)
        return (low + high) / 2
    
    return float(x)

In [55]:
ipo_withdraw_dfs['Price Per Share'] = ipo_withdraw_dfs['Price Range'].apply(split_price_range)
ipo_withdraw_dfs['Shares Offered'] = ipo_withdraw_dfs['Shares Offered'].apply(split_price_range)
ipo_withdraw_dfs['IPO Price'] = ipo_withdraw_dfs['Price Per Share'] * ipo_withdraw_dfs['Shares Offered']

In [79]:
ipo_price_order = ipo_withdraw_dfs.sort_values('IPO Price', ascending=False).reset_index(drop=True)
ipo_price_order['IPO Price'] = round(ipo_price_order['IPO Price'] / 1000000, 2).astype(str) + 'M'

max_ipo_price = ipo_price_order.loc[0]
max_ipo_price

Symbol                         NVL
Company Name          Novelis Inc.
Price Range        $18.00 - $21.00
Shares Offered          45000000.0
Price Per Share               19.5
IPO Price                   877.5M
Name: 0, dtype: object

### Question 2: Median Sharpe Ratio for 2024 IPOs (First 5 Months)
What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

Data Source: Using the same approach as in Question 1, download the IPOs in 2024 from:
https://stockanalysis.com/ipos/2024/

#### Definition of terms
* **Sharpe Ratio:** this is a financial metric used to evaluate the risk-adjusted return of an investment. It tells you how much excess return you’re getting for the extra volatility you endure by holding a riskier asset.

In [109]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_stock = pd.read_html(response.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\2197623396.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_stock = pd.read_html(response.text)


In [110]:
ipo_stock_df = ipo_stock[0]

ipo_stock_df.head(2)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$3.55,-12.25%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$18.32,346.00%


In [111]:
ipo_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [119]:
# change data type
# remove dollar sign
def clean_special_chars(cols):
    for c in cols:
        ipo_stock_df[c] = ipo_stock_df[c].apply(lambda x: 0 if x == '-' else x)
        if c == 'Return':
            ipo_stock_df[c] = ipo_stock_df[c].str.replace('%', '')
        else:
            ipo_stock_df[c] = ipo_stock_df[c].str.replace('$', '')
        

def change_type_float(cols):
    for c in cols:
        ipo_stock_df[c] = ipo_stock_df[c].astype(float)

def change_type_date(col):
    ipo_stock_df[col] = pd.to_datetime(ipo_stock_df[col])


In [120]:
# clean cols
cols = ['IPO Price', 'Current', 'Return']
clean_special_chars(cols)
change_type_float(cols)
change_type_date('IPO Date')

In [124]:
trim_ipo_stock_df = ipo_stock_df[ipo_stock_df['IPO Date'] < '2024-06-01'].reset_index(drop=True)
trim_ipo_stock_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-31,NAKA,"Kindly MD, Inc.",NaN,14.18,NaN
1,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,17.00,35.09,114.06
2,2024-05-17,HDL,Super Hi International Holding Ltd.,19.56,20.41,4.50
3,2024-05-17,RFAI,RF Acquisition Corp II,10.00,10.50,5.10
4,2024-05-15,JDZG,JIADE Limited,4.00,0.27,-92.63


In [126]:
trim_ipo_stock_df.shape

(77, 6)